In [1]:
include(joinpath("..","src","setup.jl"))

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/Compose.jl:164
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/mapping.jl:228


RObject{VecSxp}
$rcalljl_options
$rcalljl_options$height
[1] 360

$rcalljl_options$width
[1] 432




In [2]:
datadir = joinpath("..","data","count_lengths","run_2018-11-26")

"../data/count_lengths/run_2018-11-26"

In [3]:
params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)
settings = joinpath("..","src","settings.toml")

results = []
for_results_in(joinpath(datadir,"data"),reinterpret="reinterpret") do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);
head(df)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


,length,percepts,created,pindex
,Float64,Int64,DateTime,Int64
1,0.2,2,2018-11-26T17:38:30.128,1
2,11.4,1,2018-11-26T17:38:30.128,1
3,0.4,2,2018-11-26T17:38:30.128,1
4,5.1,1,2018-11-26T17:38:30.128,1
5,0.4,2,2018-11-26T17:38:30.128,1
6,18.6,1,2018-11-26T17:38:30.128,1


In [4]:
hstr,hlen = human_error_by_sid(resample=1000,N=N_for_pressnitzer_hupe_2006);

In [5]:
quantile(hlen.x1,(0.025,0.5,0.975))

(0.11324200913242048, 0.20684931506849324, 0.3690182648401826)

In [6]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ]
progress = Progress(nrow(unique(params[fields])))
df_summary = by(params,fields) do row
    next!(progress)
    err = model_error(df,params;(k => row[1,k] for k in fields)...)
    stt = model_stream_stats(df,params;(k => row[1,k] for k in fields)...)
    DataFrame(stream_error = err.stream,length_error = err.lengths,
              error_diff = err.lengths - err.stream,
              stream_ratio = err.stream / err.lengths,
              stream_spread = stt.spread,
              stream_shift = stt.shift,
              eratio = error_ratio(err,human_error(N=N_for_pressnitzer_hupe_2006)))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:20


,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,stream_error,length_error,error_diff,stream_ratio,stream_spread,stream_shift,eratio
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.474594,0.415494,-0.0591007,1.14224,-0.491088,-1.26919,2.89455
2,0.0,5.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.261357,0.30013,0.0387728,0.870813,0.24173,-0.442544,1.73967
3,0.0,15.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.265539,0.388889,0.12335,0.682815,0.223439,0.421181,1.98618
4,0.0,44.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.262928,1.0,0.737072,0.262928,0.11994,0.534381,3.3644
5,0.0,130.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.26317,1.0,0.73683,0.26317,0.110914,0.543407,3.40041
6,0.0,390.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.262932,1.0,0.737068,0.262932,0.111201,0.54312,3.37441
7,0.0,1100.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.27456,0.720018,0.445457,0.381324,0.14993,0.358554,2.77986
8,0.0,3400.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.263395,1.0,0.736605,0.263395,0.103843,0.550478,3.41026
9,0.0,10000.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.264263,1.0,0.735737,0.264263,0.138839,0.515482,3.40651


In [7]:
dfplot_within = @linq df_summary |>
    rename_levels_for([:stream_error,:length_error,:eratio]) |>
    transform(a = packaxes(:c_a,pad=0.25), m = packaxes(:c_m,pad=0.25), rand = randn(length(:c_a))) |>
    transform(eratio_str = fmt.("01.1f",round.(:eratio,digits=1)))
              
head(dfplot_within)

,c_a,c_m,level,stream_error,length_error,eratio,a,m,rand,eratio_str
,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,Peripheral,0.474594,0.415494,2.89455,0.0,0.0,1.15586,2.9
2,0.0,5.0,Peripheral,0.261357,0.30013,1.73967,0.0,1.25,1.12574,1.7
3,0.0,15.0,Peripheral,0.265539,0.388889,1.98618,0.0,2.25,0.309853,2.0
4,0.0,44.0,Peripheral,0.262928,1.0,3.3644,0.0,3.25,-0.238329,3.4
5,0.0,130.0,Peripheral,0.26317,1.0,3.40041,0.0,4.25,-0.0927928,3.4
6,0.0,390.0,Peripheral,0.262932,1.0,3.37441,0.0,5.25,0.0713984,3.4


In [8]:
datadir_all = joinpath("..","data","count_lengths","run_2018-11-28")
datadir_selected = joinpath("..","data","count_lengths","run_2018-11-30")

"../data/count_lengths/run_2018-11-30"

In [9]:
params_all = load_params(joinpath(datadir_all,"params.jld2"))
params_all[:pindex] = 1:size(params_all,1)
settings = joinpath(srcdir,"settings.toml")

results = []
for_results_in(joinpath(datadir_all,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df_all = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:29


In [10]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ]
progress = Progress(nrow(unique(params_all[fields])))
herr = human_error(resample=10_000,N=N_for_pressnitzer_hupe_2006)
df_summary_all = by(params_all,fields) do row
    next!(progress)
    err = model_error(df_all,params_all;(k => row[1,k] for k in fields)...)
    DataFrame(stream_error = err.stream,length_error = err.lengths,eratio = error_ratio(err,herr),pindex = row.pindex[1])
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:05


,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,stream_error,length_error,eratio,pindex
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.306539,0.460878,2.2808,1
2,0.0,5.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.291946,0.357271,1.98512,2
3,0.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.254462,1.0,3.30781,3
4,0.0,790.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.254462,1.0,3.30781,4
5,0.0,10000.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.254462,1.0,3.30781,5
6,0.0,0.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.302625,0.417442,2.16565,6
7,0.0,5.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.301851,0.417123,2.16182,7
8,0.0,63.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.254462,1.0,3.30781,8
9,0.0,790.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.254462,1.0,3.30781,9


In [11]:
params_selected = load_params(joinpath(datadir_selected,"params.jld2"))
params_selected[:pindex] = 1:size(params_selected,1)
settings = joinpath(srcdir,"settings.toml")

results = []
for_results_in(joinpath(datadir_selected,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df_selected = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


In [12]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ]
progress = Progress(nrow(unique(params_selected[fields])))
df_summary_selected = by(params_selected,fields) do row
    next!(progress)
    err = model_error(df_selected,params_selected;(k => row[1,k] for k in fields)...)
    DataFrame(stream_error = err.stream,length_error = err.lengths, eratio = error_ratio(err,herr),pindex = row.pindex[1])
end

Progress:  86%|███████████████████████████████████      |  ETA: 0:00:01┌ Warning: Expected 3 parameter entries. 
│ Instead, only found entires: 2×36 DataFrame
│ │ Row │ f_W_m_σ │ f_c_m   │ t_c_a   │ t_c_m   │ f         │ t_W_m_c │ t_c_x   │ f_c_x   │ t_W_m_σ_t │ s_τ_σ     │ f_c_a   │ f_c_σ   │ s_c_m   │ f_τ_a     │ s_c_σ   │ Δt        │ s_τ_m     │ s_c_x   │ t_W_m_σ_ϕ │ t_τ_a     │ s_c_a   │ t_τ_m     │ t_c_σ   │ t_τ_x     │ s_W_m_c │ s_τ_a     │ f_W_m_c │ t_τ_σ     │ f_τ_σ     │ f_τ_x     │ Δf    │ t_W_m_σ_N │ f_τ_m     │ s_τ_x     │ s_W_m_σ │ pindex │
│ │     │ Float64 │ Float64 │ Float64 │ Float64 │ Quantity… │ Float64 │ Float64 │ Float64 │ Float64   │ Quantity… │ Float64 │ Float64 │ Float64 │ Quantity… │ Float64 │ Quantity… │ Quantity… │ Float64 │ Float64   │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Quantity… │ Quantity… │ Int64 │ Float64   │ Quantity… │ Quantity… │ Float64 │ Int64  │
│ ├─────┼─────────┼─────────┼─────────┼

,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,stream_error,length_error,eratio,pindex
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,5.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.133125,0.279565,1.17245,1
2,5.0,5.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.43188,0.27089,2.34641,2
3,5.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.135811,0.185763,0.968284,3
4,5.0,0.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.150671,0.316943,1.32819,4
5,5.0,5.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.432949,0.316622,2.45545,5
6,5.0,63.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.136061,0.194819,0.990032,6
7,5.0,0.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,0.146599,0.261469,1.18483,7
8,5.0,5.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,0.423751,0.291239,2.36054,8
9,5.0,63.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,0.132938,0.199922,0.989243,9


In [13]:
# if there is a selected entry with a revised eratio, use that value

df_summary_selected[:procedure] = "selected"
df_summary_all[:procedure] = "survey"
function favor_selected(vals,procedure)
    selected = findfirst(procedure .== "selected")
    if selected isa Nothing
        @assert length(vals) == 1
        first(vals)
    else
        vals[selected]
    end
end
    
df_summary_multi = @linq vcat(df_summary_all,df_summary_selected) |>
    by(fields,eratio = favor_selected(:eratio,:procedure),
              stream_error = favor_selected(:stream_error,:procedure),
              length_error = favor_selected(:length_error,:procedure))
head(df_summary_multi)

,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,eratio,stream_error,length_error
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,2.2808,0.306539,0.460878
2,0.0,5.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,1.98512,0.291946,0.357271
3,0.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.30781,0.254462,1.0
4,0.0,790.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.30781,0.254462,1.0
5,0.0,10000.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.30781,0.254462,1.0
6,0.0,0.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,2.16565,0.302625,0.417442


In [14]:
mlevels = [(vars = [:f_c_a,:f_c_m], name = "Peripheral"),
          (vars = [:s_c_a,:s_c_m], name = "Cortical"),
          (vars = [:t_c_a,:t_c_m], name = "Object")]
dflevels = map(mlevels) do level
    result = by(df_summary_multi,level.vars) do slice
       DataFrame(c_a = first(slice[level.vars[1]]),c_m = first(slice[level.vars[2]]),
                 level = level.name, 
                 eratio = minimum(skipmissing(slice.eratio)),
                 stream_error = slice.stream_error[argmin(coalesce.(slice.eratio,-Inf))],
                 length_error = slice.length_error[argmin(coalesce.(slice.eratio,-Inf))])
            
    end
    result[[:c_a,:c_m,:level,:eratio,:stream_error,:length_error]]
end

3-element Array{DataFrame,1}:
 25×6 DataFrame. Omitted printing of 1 columns
│ Row │ c_a     │ c_m     │ level      │ eratio   │ stream_error │
│     │ Float64 │ Float64 │ String     │ Float64  │ Float64      │
├─────┼─────────┼─────────┼────────────┼──────────┼──────────────┤
│ 1   │ 0.0     │ 0.0     │ Peripheral │ 1.23054  │ 0.194559     │
│ 2   │ 0.0     │ 5.0     │ Peripheral │ 0.873774 │ 0.115321     │
│ 3   │ 0.0     │ 63.0    │ Peripheral │ 1.23513  │ 0.18868      │
│ 4   │ 0.0     │ 790.0   │ Peripheral │ 1.01545  │ 0.146695     │
│ 5   │ 0.0     │ 10000.0 │ Peripheral │ 1.0704   │ 0.155903     │
│ 6   │ 5.0     │ 0.0     │ Peripheral │ 0.835578 │ 0.106877     │
│ 7   │ 5.0     │ 5.0     │ Peripheral │ 1.32171  │ 0.215109     │
│ 8   │ 5.0     │ 63.0    │ Peripheral │ 0.627834 │ 0.092224     │
│ 9   │ 5.0     │ 790.0   │ Peripheral │ 1.06475  │ 0.144573     │
│ 10  │ 5.0     │ 10000.0 │ Peripheral │ 1.49245  │ 0.239424     │
⋮
│ 15  │ 63.0    │ 10000.0 │ Peripheral │ 1.50687  

In [15]:
dfplot_across = @linq vcat(dflevels...) |>
    transform(a = packaxes(:c_a,maxpad=false), m = packaxes(:c_m,maxpad=false)) |>
    transform(eratio_str = fmt.("01.2f",round.(:eratio,digits=2)))
head(dfplot_across)

,c_a,c_m,level,eratio,stream_error,length_error,a,m,eratio_str
,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,Peripheral,1.23054,0.194559,0.197766,0.0,0.0,1.23
2,0.0,5.0,Peripheral,0.873774,0.115321,0.180251,0.0,1.5,0.87
3,0.0,63.0,Peripheral,1.23513,0.18868,0.210022,0.0,2.5,1.24
4,0.0,790.0,Peripheral,1.01545,0.146695,0.187367,0.0,3.5,1.02
5,0.0,10000.0,Peripheral,1.0704,0.155903,0.195292,0.0,4.5,1.07
6,5.0,0.0,Peripheral,0.835578,0.106877,0.178306,1.5,0.0,0.84


In [16]:
delete!(dfplot_within,:rand)

,c_a,c_m,level,stream_error,length_error,eratio,a,m,eratio_str
,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,Peripheral,0.474594,0.415494,2.89455,0.0,0.0,2.9
2,0.0,5.0,Peripheral,0.261357,0.30013,1.73967,0.0,1.25,1.7
3,0.0,15.0,Peripheral,0.265539,0.388889,1.98618,0.0,2.25,2.0
4,0.0,44.0,Peripheral,0.262928,1.0,3.3644,0.0,3.25,3.4
5,0.0,130.0,Peripheral,0.26317,1.0,3.40041,0.0,4.25,3.4
6,0.0,390.0,Peripheral,0.262932,1.0,3.37441,0.0,5.25,3.4
7,0.0,1100.0,Peripheral,0.27456,0.720018,2.77986,0.0,6.25,2.8
8,0.0,3400.0,Peripheral,0.263395,1.0,3.41026,0.0,7.25,3.4
9,0.0,10000.0,Peripheral,0.264263,1.0,3.40651,0.0,8.25,3.4


In [17]:
df_bar_plot = @linq vcat(dfplot_within,dfplot_across) |>
    by(:level,prop = mean(:eratio .<= 1.0))

,level,prop
,String,Float64
1,Peripheral,0.048
2,Cortical,0.152
3,Object,0.296


In [23]:
p = plot(df_bar_plot,x=:level,y=:prop,Geom.bar,Theme(bar_spacing=0.5inch))
draw(SVG(6inch,4inch),p)
draw(PDF(joinpath(plotdir,"..","barplot.pdf"),6inch,4inch),p)

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [19]:
df_bar_plot2 = @linq dfplot_within |>
    by(:level,prop = mean(:eratio .<= 1.0))

,level,prop
,String,Float64
1,Peripheral,0.02
2,Cortical,0.06
3,Object,0.12


In [24]:
p = plot(df_bar_plot2,x=:level,y=:prop,Geom.bar,Theme(bar_spacing=0.2inch))
draw(SVG(3inch,2inch),p)
draw(PDF(joinpath(grantdir,"fig3_raw.pdf"),3inch,2inch),p)

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
